# Poelis Python SDK Tutorial

Welcome to the Poelis Python SDK tutorial! This notebook will guide you through setting up and using the SDK to interact with the Poelis API.

## What is Poelis?

Poelis is a GraphQL-first platform for managing hierarchical data structures, products, items, and their properties. The Python SDK provides a convenient way to interact with the Poelis API from your Python applications.

## Table of Contents

1. [Installation](#Installation)
2. [Authentication Setup](#Authentication-Setup)
3. [Basic Client Setup](#Basic-Client-Setup)
4. [Exploring Workspaces](#Exploring-Workspaces)
5. [Working with Products](#Working-with-Products)
6. [Managing Items](#Managing-Items)
7. [Search Functionality](#Search-Functionality)
8. [Browser Interface](#Browser-Interface)
9. [Error Handling](#Error-Handling)
10. [Best Practices](#Best-Practices)


## 1. Installation

First, let's install the Poelis Python SDK. The SDK requires Python 3.11 or higher.

```bash
pip install poelis-sdk
```

### Requirements
- Python >= 3.11
- Access to a Poelis API endpoint
- Valid API credentials (API key + organization ID)


In [2]:
!pip install poelis-sdk

The folder you are executing pip from can no longer be found.


In [ ]:
# Import the SDK
from poelis_sdk import PoelisClient

# Check if the SDK is properly installed
print("Poelis SDK imported successfully!")
print(f"PoelisClient class: {PoelisClient}")


## 2. Authentication Setup

Before you can use the SDK, you need to obtain your API credentials from the Poelis web interface.

### Getting Your Credentials

1. **Navigate to Organization Settings → API Keys** in the Poelis web interface
2. **Create a new API key**:
   - Choose a descriptive name
   - Select appropriate scopes (read-only recommended for beginners)
   - Copy the key immediately (it's only shown once!)
3. **Note your Organization ID** (displayed in the same section)

### Environment Variables (Recommended)

For security, store your credentials as environment variables:

```bash
export POELIS_API_KEY=poelis_live_A1B2C3...
export POELIS_ORG_ID=tenant_uci_001
# POELIS_BASE_URL is optional - defaults to https://api.poelis.ai
```

### Base URLs

The SDK defaults to the production API (`https://api.poelis.ai`). You can override this for different environments:

- **Local development**: `base_url="http://localhost:8000"`
- **Staging**: `base_url="https://api.staging.poelis.ai"` (example)
- **Production**: No need to specify, uses `https://api.poelis.ai` by default

Confirm the exact URLs for your specific deployment.


## 3. Basic Client Setup

There are two ways to initialize the Poelis client:

### Method 1: Direct Initialization (Simplified)

```python
# Simple usage with defaults
client = PoelisClient(
    api_key="poelis_live_A1B2C3...",
    org_id="tenant_uci_001"
    # base_url defaults to https://api.poelis.ai
)

# Override for different environments
client = PoelisClient(
    api_key="poelis_live_A1B2C3...",
    org_id="tenant_uci_001",
    base_url="http://localhost:8000"  # for local development
)
```

### Method 2: From Environment Variables (Recommended)

```python
client = PoelisClient.from_env()
```

This method automatically reads the `POELIS_API_KEY` and `POELIS_ORG_ID` environment variables. The `POELIS_BASE_URL` environment variable is optional and defaults to the production API.


In [ ]:
# Example: Initialize client (replace with your actual credentials)
# For this demo, we'll use placeholder values
# In practice, use environment variables or replace with your actual credentials

try:
    # Try to initialize from environment variables first
    client = PoelisClient.from_env()
    print("✅ Client initialized from environment variables")
    print(f"Base URL: {client.base_url}")
    print(f"Organization ID: {client.org_id}")
except ValueError as e:
    print(f"❌ Environment variables not set: {e}")
    print("Please set POELIS_API_KEY and POELIS_ORG_ID (POELIS_BASE_URL is optional)")
    
    # Fallback to manual initialization (replace with your credentials)
    # client = PoelisClient(
    #     api_key="your_api_key_here",
    #     org_id="your_org_id_here"
    #     # base_url will default to https://api.poelis.ai
    # )


## 4. Exploring Workspaces

Workspaces are the top-level organizational units in Poelis. Let's explore what workspaces are available in your organization.


In [ ]:
# List all workspaces
try:
    workspaces = client.workspaces.list(limit=10, offset=0)
    print(f"Found {len(workspaces)} workspaces:")
    
    for i, workspace in enumerate(workspaces, 1):
        print(f"{i}. ID: {workspace.get('id', 'N/A')}")
        print(f"   Name: {workspace.get('name', 'N/A')}")
        print(f"   Description: {workspace.get('description', 'N/A')}")
        print()
        
    # Get the first workspace for further examples
    if workspaces:
        first_workspace = workspaces[0]
        workspace_id = first_workspace['id']
        print(f"Using workspace '{first_workspace.get('name', workspace_id)}' for examples")
    else:
        print("No workspaces found. You may need to create one in the web interface.")
        
except Exception as e:
    print(f"Error listing workspaces: {e}")
    print("Make sure your credentials are correct and you have access to workspaces.")


## 5. Working with Products

Products are collections of items within a workspace. Let's explore how to work with products.


In [ ]:
# List products in a workspace
try:
    # Use the workspace_id from the previous example, or replace with your workspace ID
    if 'workspace_id' in locals():
        products_page = client.products.list_by_workspace(
            workspace_id=workspace_id, 
            limit=10, 
            offset=0
        )
        
        print(f"Found {len(products_page.data)} products in workspace '{workspace_id}':")
        
        for i, product in enumerate(products_page.data, 1):
            print(f"{i}. ID: {product.id}")
            print(f"   Name: {product.name}")
            print(f"   Code: {getattr(product, 'code', 'N/A')}")
            print()
            
        # Get the first product for further examples
        if products_page.data:
            first_product = products_page.data[0]
            product_id = first_product.id
            print(f"Using product '{first_product.name}' for examples")
        else:
            print("No products found in this workspace.")
    else:
        print("No workspace available. Please run the workspace exploration cell first.")
        
except Exception as e:
    print(f"Error listing products: {e}")
    print("Make sure you have a valid workspace ID and appropriate permissions.")


In [ ]:
# Alternative: List all products across all workspaces
try:
    print("All products across all workspaces:")
    print("=" * 50)
    
    # Use the iterator to get all products
    product_count = 0
    for product in client.products.iter_all(page_size=10):
        product_count += 1
        print(f"{product_count}. {product.name} (ID: {product.id})")
        
        # Limit output for demo purposes
        if product_count >= 20:
            print("... (showing first 20 products)")
            break
            
    print(f"\nTotal products found: {product_count}")
    
except Exception as e:
    print(f"Error iterating through products: {e}")


## 6. Managing Items

Items are the individual entities within products. Let's explore how to work with items.


In [ ]:
# List items in a product
try:
    if 'product_id' in locals():
        items_page = client.items.list_by_product(
            product_id=product_id, 
            limit=10, 
            offset=0
        )
        
        print(f"Found {len(items_page['data'])} items in product '{product_id}':")
        
        for i, item in enumerate(items_page['data'], 1):
            print(f"{i}. ID: {item.get('id', 'N/A')}")
            print(f"   Name: {item.get('name', 'N/A')}")
            print(f"   Code: {item.get('code', 'N/A')}")
            print()
            
        # Get the first item for further examples
        if items_page['data']:
            first_item = items_page['data'][0]
            item_id = first_item['id']
            print(f"Using item '{first_item.get('name', item_id)}' for examples")
        else:
            print("No items found in this product.")
    else:
        print("No product available. Please run the product exploration cell first.")
        
except Exception as e:
    print(f"Error listing items: {e}")
    print("Make sure you have a valid product ID and appropriate permissions.")


In [ ]:
# Get details of a specific item
try:
    if 'item_id' in locals():
        item_details = client.items.get(item_id)
        
        print(f"Item Details for '{item_id}':")
        print("=" * 40)
        
        for key, value in item_details.items():
            print(f"{key}: {value}")
    else:
        print("No item ID available. Please run the items listing cell first.")
        
except Exception as e:
    print(f"Error getting item details: {e}")


## 7. Search Functionality

The Poelis SDK provides powerful search capabilities across products, items, and properties.


In [ ]:
# Search for products
try:
    print("Searching for products:")
    print("=" * 30)
    
    # Search for products with a query
    product_search_results = client.search.products(q="*", limit=5, offset=0)
    
    print(f"Total products found: {product_search_results.get('total', 0)}")
    print(f"Showing {len(product_search_results.get('hits', []))} results:")
    
    for i, product in enumerate(product_search_results.get('hits', []), 1):
        print(f"{i}. {product.get('name', 'N/A')} (ID: {product.get('id', 'N/A')})")
        
except Exception as e:
    print(f"Error searching products: {e}")


In [ ]:
# Search for items
try:
    print("\nSearching for items:")
    print("=" * 30)
    
    # Search for items with a query
    item_search_results = client.search.items(q="*", limit=5, offset=0)
    
    print(f"Total items found: {item_search_results.get('total', 0)}")
    print(f"Showing {len(item_search_results.get('hits', []))} results:")
    
    for i, item in enumerate(item_search_results.get('hits', []), 1):
        print(f"{i}. {item.get('name', 'N/A')} (ID: {item.get('id', 'N/A')})")
        
except Exception as e:
    print(f"Error searching items: {e}")


In [ ]:
# Search for properties within a workspace
try:
    if 'workspace_id' in locals():
        print(f"\nSearching for properties in workspace '{workspace_id}':")
        print("=" * 50)
        
        # Search for properties with a query
        property_search_results = client.search.properties(
            q="*", 
            workspace_id=workspace_id, 
            limit=5, 
            offset=0
        )
        
        print(f"Total properties found: {property_search_results.get('total', 0)}")
        print(f"Showing {len(property_search_results.get('hits', []))} results:")
        
        for i, prop in enumerate(property_search_results.get('hits', []), 1):
            print(f"{i}. {prop.get('name', 'N/A')} (ID: {prop.get('id', 'N/A')})")
    else:
        print("No workspace available for property search.")
        
except Exception as e:
    print(f"Error searching properties: {e}")


## 8. Browser Interface

The SDK provides a convenient browser interface for interactive exploration of your data hierarchy.


In [ ]:
# Explore the browser interface
try:
    print("Browser Interface Demo:")
    print("=" * 30)
    
    # The browser provides dot-path navigation
    print("Available browser attributes:")
    print(f"- client.browser: {type(client.browser)}")
    
    # You can explore the hierarchy interactively
    print("\nTo explore interactively:")
    print("1. Type 'client.browser.' and press TAB for autocomplete")
    print("2. Navigate through workspaces → products → items → properties")
    print("3. Example: client.browser.<workspace>.<product>.<item>.<child>.properties")
    
    # Show what's available at the top level
    print(f"\nBrowser object: {client.browser}")
    print("Use TAB completion in Jupyter/VSCode to explore the hierarchy!")
    
except Exception as e:
    print(f"Error accessing browser interface: {e}")


## 9. Error Handling

Proper error handling is essential when working with APIs. Let's explore common error scenarios and how to handle them.


In [ ]:
# Example: Error handling patterns
import time

def safe_api_call(func, *args, **kwargs):
    """Safely call an API function with retry logic and error handling."""
    max_retries = 3
    retry_delay = 1
    
    for attempt in range(max_retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2  # Exponential backoff
            else:
                print("All retry attempts failed")
                raise

# Example usage with error handling
try:
    print("Testing error handling patterns:")
    print("=" * 40)
    
    # This will work if you have valid credentials
    workspaces = safe_api_call(client.workspaces.list, limit=5, offset=0)
    print(f"✅ Successfully retrieved {len(workspaces)} workspaces")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("This might be due to:")
    print("- Invalid API credentials")
    print("- Network connectivity issues")
    print("- API endpoint unavailability")
    print("- Insufficient permissions")


## 10. Best Practices

Here are some best practices for using the Poelis Python SDK effectively:


### Security Best Practices

1. **Use Environment Variables**: Never hardcode API keys in your code
2. **Rotate Keys Regularly**: Create new API keys and revoke old ones periodically
3. **Use Read-Only Keys**: Start with read-only permissions and only add write permissions when needed
4. **Store Keys Securely**: Use secure key management systems in production

### Performance Best Practices

1. **Use Iterators**: Use `iter_all()` methods for large datasets instead of manual pagination
2. **Set Appropriate Limits**: Use reasonable page sizes (10-100 items) to balance performance and memory usage
3. **Cache Results**: Cache frequently accessed data to reduce API calls
4. **Handle Rate Limits**: Implement exponential backoff for retry logic

### Code Organization Best Practices

1. **Error Handling**: Always wrap API calls in try-catch blocks
2. **Type Hints**: Use type hints for better code documentation and IDE support
3. **Logging**: Add logging for debugging and monitoring
4. **Configuration**: Use configuration files or environment variables for different environments


In [ ]:
# Example: Production-ready client setup
import os
import logging
from typing import Optional

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class PoelisManager:
    """Production-ready wrapper for Poelis client with best practices."""
    
    def __init__(self, base_url: Optional[str] = None, api_key: Optional[str] = None, org_id: Optional[str] = None):
        """Initialize with environment variables or explicit parameters."""
        self.base_url = base_url or os.getenv("POELIS_BASE_URL")
        self.api_key = api_key or os.getenv("POELIS_API_KEY")
        self.org_id = org_id or os.getenv("POELIS_ORG_ID")
        
        if not all([self.base_url, self.api_key, self.org_id]):
            raise ValueError("Missing required configuration. Set environment variables or provide parameters.")
        
        self.client = PoelisClient(
            base_url=self.base_url,
            api_key=self.api_key,
            org_id=self.org_id
        )
        
        logger.info(f"Poelis client initialized for org: {self.org_id}")
    
    def get_workspaces(self, limit: int = 100) -> list:
        """Get all workspaces with error handling."""
        try:
            return self.client.workspaces.list(limit=limit, offset=0)
        except Exception as e:
            logger.error(f"Failed to get workspaces: {e}")
            raise
    
    def get_products_summary(self) -> dict:
        """Get a summary of all products across workspaces."""
        try:
            workspaces = self.get_workspaces()
            summary = {
                "total_workspaces": len(workspaces),
                "workspaces": []
            }
            
            for workspace in workspaces:
                ws_id = workspace['id']
                products = self.client.products.list_by_workspace(workspace_id=ws_id, limit=100, offset=0)
                
                summary["workspaces"].append({
                    "id": ws_id,
                    "name": workspace.get('name', 'N/A'),
                    "product_count": len(products.data)
                })
            
            return summary
            
        except Exception as e:
            logger.error(f"Failed to get products summary: {e}")
            raise

# Example usage
try:
    manager = PoelisManager()
    summary = manager.get_products_summary()
    
    print("Production-ready example:")
    print("=" * 30)
    print(f"Total workspaces: {summary['total_workspaces']}")
    for ws in summary['workspaces']:
        print(f"- {ws['name']}: {ws['product_count']} products")
        
except Exception as e:
    print(f"Error in production example: {e}")
    print("This is expected if you don't have valid credentials set up.")


## Next Steps

Congratulations! You've completed the Poelis Python SDK tutorial. Here's what you can do next:

### 1. Explore the Full API
- Check out the [API Documentation](https://docs.poelis.ai) for complete endpoint details
- Explore the GraphQL schema for advanced queries
- Try the browser interface for interactive exploration

### 2. Build Your Application
- Start with read-only operations to understand your data structure
- Implement error handling and retry logic
- Add logging and monitoring
- Consider caching strategies for better performance

### 3. Advanced Features
- Explore hierarchical search capabilities
- Work with property types and dependencies
- Implement batch operations for large datasets
- Use the browser interface for interactive data exploration

### 4. Get Help
- Check the [SDK Documentation](https://github.com/poelis/poelis-python-sdk)
- Join the community discussions
- Report issues or request features

### 5. Environment Setup Checklist
Before running this notebook with real data, make sure you have:
- [ ] Python 3.11+ installed
- [ ] Poelis SDK installed (`pip install poelis-sdk`)
- [ ] Valid API credentials from Organization Settings → API Keys
- [ ] Environment variables set (`POELIS_BASE_URL`, `POELIS_API_KEY`, `POELIS_ORG_ID`)
- [ ] Network access to your Poelis API endpoint

Happy coding with Poelis! 🚀
